In [1]:
import plotly.graph_objects as go
import pandas as pd

# Parameters 2025, Vaud
H = 380_000
p_g = 0.30
p_hp = 0.17
L_g = 20
L_hp = 20
cu_g = 2.7     # kg per gas boiler
cu_hp = 36     # kg per heat pump
rr = 0.61      # end-of-life recycling rate
a = 0.80       # allocation of recovered to local manufacturing
m_loss = 0.05  # manufacturing loss
installs_per_year = H*(p_g+p_hp) / L_g   # replace gb + hp with only hp


# COMPUTE MASSES (kg)

EoL_g_units = installs_per_year                    # 4560 units
EoL_hp_units = (H * p_hp) / L_hp                   # 3230 units

GB_EoL_cu_kg = EoL_g_units * cu_g
GB_recov_kg = GB_EoL_cu_kg * rr
HP_EoL_cu_kg = EoL_hp_units * cu_hp
HP_recov_kg = HP_EoL_cu_kg * rr

secondary_available_kg = GB_recov_kg + HP_recov_kg
recycled_to_manuf_kg = secondary_available_kg * a

new_HP_demand_kg = installs_per_year * cu_hp
primary_needed_kg = max(0, new_HP_demand_kg - recycled_to_manuf_kg)
manufacturing_losses_kg = new_HP_demand_kg * m_loss

nonrecov_gb_kg = GB_EoL_cu_kg * (1 - rr)
nonrecov_hp_kg = HP_EoL_cu_kg * (1 - rr)
irrecoverable_kg = nonrecov_gb_kg + nonrecov_hp_kg + manufacturing_losses_kg

# Convert to tonnes for plotting
kg_to_t = lambda x: x / 1000
flows_t = {
    "GB_EoL_cu_t": kg_to_t(GB_EoL_cu_kg),
    "GB_recov_t": kg_to_t(GB_recov_kg),
    "HP_EoL_cu_t": kg_to_t(HP_EoL_cu_kg),
    "HP_recov_t": kg_to_t(HP_recov_kg),
    "secondary_available_t": kg_to_t(secondary_available_kg),
    "recycled_to_manuf_t": kg_to_t(recycled_to_manuf_kg),
    "new_HP_demand_t": kg_to_t(new_HP_demand_kg),
    "primary_needed_t": kg_to_t(primary_needed_kg),
    "manufacturing_losses_t": kg_to_t(manufacturing_losses_kg),
    "irrecoverable_t": kg_to_t(irrecoverable_kg),
}

# Nodes (ordered)
labels = [
    "GB EoL outflow",
    "Recovered Cu (GB)",
    "HP EoL outflow",
    "Recovered Cu (HP)",
    "Recycled Cu available",
    "Recycled → local manufacturing",
    "Primary Cu supply",
    "Manufacturing (HP assembly)",
    "Manufacturing losses",
    "New HP installations",
    "Irrecoverable losses"
]

# Build sources, targets, values (match labels indices)
label_index = {lab:i for i,lab in enumerate(labels)}
src = []
tgt = []
val = []
link_color = []

# Flows
def add_flow(s_label, t_label, tonnes, color):
    src.append(label_index[s_label])
    tgt.append(label_index[t_label])
    val.append(round(tonnes,3))
    link_color.append(color)

green = "rgba(44,160,44,0.8)"   # recycled
orange = "rgba(255,127,14,0.8)" # primary
grey = "rgba(127,127,127,0.6)"  # losses
blue = "rgba(31,119,180,0.8)"   # stock/EoL

# GB EoL -> Recovered & nonrecovered
add_flow("GB EoL outflow", "Recovered Cu (GB)", flows_t["GB_recov_t"], green)
add_flow("GB EoL outflow", "Irrecoverable losses", flows_t["GB_EoL_cu_t"] - flows_t["GB_recov_t"], grey)

# HP EoL -> Recovered & nonrecovered
add_flow("HP EoL outflow", "Recovered Cu (HP)", flows_t["HP_recov_t"], green)
add_flow("HP EoL outflow", "Irrecoverable losses", flows_t["HP_EoL_cu_t"] - flows_t["HP_recov_t"], grey)

# Recovered Cu -> Recycled available (sum)
add_flow("Recovered Cu (GB)", "Recycled Cu available", flows_t["GB_recov_t"], green)
add_flow("Recovered Cu (HP)", "Recycled Cu available", flows_t["HP_recov_t"], green)

# Recycled available -> Recycled to manuf
add_flow("Recycled Cu available", "Recycled → local manufacturing", flows_t["recycled_to_manuf_t"], green)

# Recycled -> Manufacturing and Primary -> Manufacturing
add_flow("Recycled → local manufacturing", "Manufacturing (HP assembly)", flows_t["recycled_to_manuf_t"], green)
add_flow("Primary Cu supply", "Manufacturing (HP assembly)", flows_t["primary_needed_t"], orange)

# Manufacturing -> New HPs and losses
add_flow("Manufacturing (HP assembly)", "New HP installations", flows_t["new_HP_demand_t"], blue)
add_flow("Manufacturing (HP assembly)", "Manufacturing losses", flows_t["manufacturing_losses_t"], grey)

# Optionally show EoL nodes as coming from stock (you can add stock nodes if desired)
# For plot simplicity, GB EoL outflow and HP EoL outflow are source nodes with values:
# (Plotly Sankey expects link totals to match flow balances visually.)

fig = go.Figure(data=[go.Sankey(
    node = dict(
        pad=15, thickness=18,
        label=labels,
        color=[
            "#1f77b4" if ("EoL" in lab or "New HP" in lab) else "#ffffff"
            for lab in labels
        ]
    ),
    link = dict(source=src, target=tgt, value=val, color=link_color)
)])

fig.update_layout(title_text="Copper flow Sankey — Canton of Vaud (2025) — tonnes Cu", font_size=11)
fig.show()

# -------------------------
# ENERGY & GHG summary (simple embodied factors)
# -------------------------
# Replace these with literature values if desired
E_primary_MJ_per_kg = 150
E_secondary_MJ_per_kg = 15
GHG_primary_kgCO2_per_kg = 3.0
GHG_secondary_kgCO2_per_kg = 0.3

energy_primary_MJ = primary_needed_kg * E_primary_MJ_per_kg
energy_secondary_MJ = recycled_to_manuf_kg * E_secondary_MJ_per_kg
GHG_primary_kg = primary_needed_kg * GHG_primary_kgCO2_per_kg
GHG_secondary_kg = recycled_to_manuf_kg * GHG_secondary_kgCO2_per_kg

df_env = pd.DataFrame({
    "Stream": ["Primary Cu used", "Secondary Cu used"],
    "Mass (kg)": [round(primary_needed_kg,0), round(recycled_to_manuf_kg,0)],
    "Energy (MJ)": [round(energy_primary_MJ,0), round(energy_secondary_MJ,0)],
    "Energy (GJ)": [round(energy_primary_MJ/1000,2), round(energy_secondary_MJ/1000,2)],
    "GHG (kgCO2)": [round(GHG_primary_kg,0), round(GHG_secondary_kg,0)],
    "GHG (tCO2)": [round(GHG_primary_kg/1000,3), round(GHG_secondary_kg/1000,3)]
})

print(df_env)


              Stream  Mass (kg)  Energy (MJ)  Energy (GJ)  GHG (kgCO2)  \
0    Primary Cu used   252969.0   37945379.0     37945.38     758908.0   
1  Secondary Cu used    68511.0    1027662.0      1027.66      20553.0   

   GHG (tCO2)  
0     758.908  
1      20.553  


In [2]:
copper_extraction_GHG = 3.5     # tCO2/tCu
copper_recycling_GHG = 0.3      # tCO2/tCu
electricity_demand_gb_production = 480 # kWh
average_electricity_demand_gb = 0.00264 # (15kW boiler)
emissions_to_air_gb = 0.056 # (15 kW boiler)

hp_energy_intensity = 

SyntaxError: invalid syntax (4182798587.py, line 7)

In [ ]:
import numpy as np
import plotly.graph_objects as go

# -------------------------
# PARAMETERS (your values)
# -------------------------
H = 380_000
p_g = 0.30
p_hp = 0.17
L_g = 20
L_hp = 20
cu_g = 2.7     # kg per gas boiler
cu_hp = 36     # kg per heat pump
rr = 0.61      # end-of-life recycling rate
a = 0.80       # allocation of recovered to local manufacturing
m_loss = 0.05  # (not used here; kept if you want)

# -------------------------
# STOCKS & FLOWS (kg)
# -------------------------
GB_stock = H * p_g                     # gas boilers in stock
HP_stock = H * p_hp                    # heat pumps in stock

GB_EoL_units = GB_stock / L_g          # annual GB outflow (units)
HP_EoL_units = HP_stock / L_hp         # annual HP outflow (units)

# Copper at EoL
GB_EoL_cu_kg = GB_EoL_units * cu_g
HP_EoL_cu_kg = HP_EoL_units * cu_hp

# Recovered copper (kg)
GB_recov_kg = GB_EoL_cu_kg * rr
HP_recov_kg = HP_EoL_cu_kg * rr

# Recycled copper available to local manufacturing (kg)
recycled_total_kg = (GB_recov_kg + HP_recov_kg) * a

# Demand: new HPs are built to replace both GB and HP (all replaced by HP)
installs_per_year = H * (p_g + p_hp) / L_g   # you used L_g earlier to replace both by HP
new_HP_demand_kg = installs_per_year * cu_hp

# Primary needed (kg)
primary_needed_kg = max(0, new_HP_demand_kg - recycled_total_kg)

# Convert to tonnes for plotting (more readable units)
kg_to_t = lambda x: x / 1000.0
GB_EoL_cu_t = kg_to_t(GB_EoL_cu_kg)
HP_EoL_cu_t = kg_to_t(HP_EoL_cu_kg)
GB_recov_t = kg_to_t(GB_recov_kg)
HP_recov_t = kg_to_t(HP_recov_kg)
recycled_total_t = kg_to_t(recycled_total_kg)
new_HP_demand_t = kg_to_t(new_HP_demand_kg)
primary_needed_t = kg_to_t(primary_needed_kg)

# -------------------------
# NODES (represent physical processes + stocks)
# -------------------------
labels = [
    "Gas boilers (stock)",
    "GB EoL outflow",
    "Recovered Cu (GB)",
    "Heat pumps (stock)",
    "HP EoL outflow",
    "Recovered Cu (HP)",
    "Recycled copper",
    "Primary Cu supply",
    "Manufacturing (new HP)",
    "New HP installations"
]

# map labels to indices
idx = {lab: i for i, lab in enumerate(labels)}

# -------------------------
# LINKS (source, target, value_tonnes)
# -------------------------
src, tgt, val, color = [], [], [], []

def add(s, t, tonnes, col):
    src.append(idx[s])
    tgt.append(idx[t])
    val.append(round(tonnes, 3))
    color.append(col)

# colors
col_stock = "rgba(31,119,180,0.8)"   # blue
col_recov = "rgba(44,160,44,0.8)"    # green
col_primary = "rgba(255,127,14,0.8)" # orange
col_manuf = "rgba(127,127,127,0.6)"  # grey

# Stocks -> EoL (show the annual outflow from stock)
add("Gas boilers (stock)", "GB EoL outflow", round(GB_EoL_cu_t,3), col_stock)      # note: units here are tonnes of Cu equivalent (for visual)
add("Heat pumps (stock)", "HP EoL outflow", round(HP_EoL_cu_t,3), col_stock)

# EoL -> Recovered Cu
# For clarity we send the copper mass at EoL to recovered nodes (already rr applied)
add("GB EoL outflow", "Recovered Cu (GB)", GB_recov_t, col_recov)
add("HP EoL outflow", "Recovered Cu (HP)", HP_recov_t, col_recov)

# Recovered -> Recycled (single unified node)
add("Recovered Cu (GB)", "Recycled copper", GB_recov_t, col_recov)
add("Recovered Cu (HP)", "Recycled copper", HP_recov_t, col_recov)

# Recycled -> Manufacturing
add("Recycled copper", "Manufacturing (new HP)", recycled_total_t, col_recov)

# Primary -> Manufacturing
add("Primary Cu supply", "Manufacturing (new HP)", primary_needed_t, col_primary)

# Manufacturing -> New HP installations
add("Manufacturing (new HP)", "New HP installations", new_HP_demand_t, col_manuf)

# New HPs -> HP stock (close the loop: new installations increase HP stock)
# This link shows that manufactured HPs become stock (closed material loop visually)
add("New HP installations", "Heat pumps (stock)", new_HP_demand_t, col_stock)

# -------------------------
# CIRCULAR LAYOUT (positions around a ring)
# -------------------------
N = len(labels)
angles = np.linspace(0, 2*np.pi, N, endpoint=False)
radius = 0.40
x = 0.5 + radius * np.cos(angles)
y = 0.5 + radius * np.sin(angles)

# Slight manual reorder to make visual loop clearer (optional)
# If you want a different grouping, change angles order manually.

node_colors = []
for lab in labels:
    if "stock" in lab or "EoL" in lab:
        node_colors.append("rgba(31,119,180,0.9)")
    elif "Recovered" in lab or "Recycled" in lab:
        node_colors.append("rgba(44,160,44,0.9)")
    elif "Primary" in lab:
        node_colors.append("rgba(255,127,14,0.9)")
    else:
        node_colors.append("white")

fig = go.Figure(data=[go.Sankey(
    arrangement="fixed",
    node = dict(
        pad = 15,
        thickness = 18,
        line = dict(color="black", width=0.5),
        label = labels,
        color = node_colors,
        x = x.tolist(),
        y = y.tolist()
    ),
    link = dict(
        source = src,
        target = tgt,
        value = val,
        color = color
    )
)])

fig.update_layout(title_text="Copper loop — Vaud 2025: stock → EoL → recovery → recycled → manufacturing → new HP → stock", font_size=11)
fig.show()

# -------------------------
# PRINT SUMMARY (tonnes)
# -------------------------
print("Summary (tonnes):")
print(f"GB EoL Cu (t): {GB_EoL_cu_t:.2f}")
print(f"HP EoL Cu (t): {HP_EoL_cu_t:.2f}")
print(f"Recovered GB (t): {GB_recov_t:.2f}")
print(f"Recovered HP (t): {HP_recov_t:.2f}")
print(f"Recycled available to manuf (t): {recycled_total_t:.2f}")
print(f"New HP demand (t): {new_HP_demand_t:.2f}")
print(f"Primary needed (t): {primary_needed_t:.2f}")



Summary (tonnes):
GB EoL Cu (t): 15.39
HP EoL Cu (t): 116.28
Recovered GB (t): 9.39
Recovered HP (t): 70.93
Recycled available to manuf (t): 64.25
New HP demand (t): 321.48
Primary needed (t): 257.23


In [ ]:
import plotly.graph_objects as go


labels = [
    "Imports",       # 0
    "GB Stock",      # 1
    "Cu",            # 2
    "HP Stock",      # 3
    "Outflow"        # 4
]

# Example flows 
flow_imports_to_cu = 50
flow_gb_to_cu = 20
flow_cu_to_hp = 60
flow_hp_to_cu = 15      # circular recovered flow
flow_hp_to_outflow = 10 # unrecoverable flow

sources = [
    0,  # Imports -> Cu
    1,  # GB Stock -> Cu
    2,  # Cu -> HP Stock
    3,  # HP Stock -> Cu (circular flow)
    3   # HP Stock -> Outflow
]

targets = [
    2,  # Cu
    2,  # Cu
    3,  # HP Stock
    2,  # Cu (circularity)
    4   # Outflow
]

values = [
    flow_imports_to_cu,
    flow_gb_to_cu,
    flow_cu_to_hp,
    flow_hp_to_cu,
    flow_hp_to_outflow
]

colors = [
    "rgba(0,100,255,0.6)",   # Imports -> Cu
    "rgba(0,150,200,0.6)",   # GB -> Cu
    "rgba(0,200,120,0.6)",   # Cu -> HP
    "rgba(0,180,80,0.6)",    # HP -> Cu (recycled)
    "rgba(255,120,50,0.6)"   # HP -> Outflow
]

# -------------------------------------------------
# Fixed positions for 4 layers 
node_x = [0.05, 0.05, 0.35, 0.65, 0.90]
node_y = [0.2, 0.7, 0.45, 0.45, 0.45]

fig = go.Figure(data=[go.Sankey(
    arrangement="fixed",
    node=dict(
        pad=20,
        thickness=22,
        line=dict(color="black", width=0.5),
        label=labels,
        x=node_x,
        y=node_y,
        color="rgba(80,80,80,0.4)"
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        color=colors
    )
)])

fig.update_layout(
    title_text="Copper Sankey for Heating Transition: Imports → Cu → HP Stock → Outflow (with Circularity)",
    font_size=14
)

fig.show()
